# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.core import Model

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice
import uuid
from azureml.core.model import InferenceConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import joblib

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_run'

experiment=Experiment(ws, experiment_name)

key = "forestfire"
dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes_mapped
count,243.000000,243.000000,243.0,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000
mean,15.761317,7.502058,2012.0,32.152263,62.041152,15.493827,0.762963,77.842387,14.680658,49.430864,4.742387,16.690535,7.035391,0.563786
std,8.842552,1.114793,0.0,3.628039,14.828160,2.811385,2.003207,14.349641,12.393040,47.665606,4.154234,14.228421,7.440568,0.496938
min,1.000000,6.000000,2012.0,22.000000,21.000000,6.000000,0.000000,28.600000,0.700000,6.900000,0.000000,1.100000,0.000000,0.000000
25%,8.000000,7.000000,2012.0,30.000000,52.500000,14.000000,0.000000,71.850000,5.800000,12.350000,1.400000,6.000000,0.700000,0.000000
50%,16.000000,8.000000,2012.0,32.000000,63.000000,15.000000,0.000000,83.300000,11.300000,33.100000,3.500000,12.400000,4.200000,1.000000
75%,23.000000,8.000000,2012.0,35.000000,73.500000,17.000000,0.500000,88.300000,20.800000,69.100000,7.250000,22.650000,11.450000,1.000000
max,31.000000,9.000000,2012.0,42.000000,90.000000,29.000000,16.800000,96.000000,65.900000,220.400000,19.000000,68.000000,31.100000,1.000000


## Create or Attach an AmlCompute cluster

In [4]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
df.head(5)

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes_mapped
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,0
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,0
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,0
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,0
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,0


In [6]:
y = df["Classes_mapped"]
X = df.drop(["Classes_mapped"], axis =1)

In [7]:
# TODO: Put your automl settings here

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : "accuracy"
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Classes_mapped",
                             featurization="auto",
                             **automl_settings
                            )


In [8]:
# TODO: Submit your experiment
run = experiment.submit(automl_config)
run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_459e811c-00f4-4099-a31f-a186a9f4de26,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_459e811c-00f4-4099-a31f-a186a9f4de26,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbal

{'runId': 'AutoML_459e811c-00f4-4099-a31f-a186a9f4de26',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-12T08:14:25.236334Z',
 'endTimeUtc': '2022-08-12T08:37:12.612559Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_run","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-204069","workspace_name":"quick-starts-ws-204069","region":"southcentralus","compute_target":"automl-clust

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.43.0, current version:1.42.0
Package:azureml-core, training version:1.43.0, current version:1.42.0
Package:azureml-dataset-runtime, training version:1.43.0, current version:1.42.0
Package:azureml-defaults, training version:1.43.0, current version:1.42.0
Package:azureml-interpret, training version:1.43.0, current version:1.42.0
Package:azureml-mlflow, training version:1.43.0.post1, current version:1.42.0
Package:azureml-pipeline-core, training version:1.43.0, current version:1.42.0
Package:azureml-responsibleai, training version:1.43.0, current version:1.42.0
Package:azureml-telemetry, training version:1.43.0, current version:1.42.0
Package:azureml-train-automl-client, training version:1.43.0, current version:1.42.0
Package:azureml-train-automl-runtime, training version:1.43.0.post1, current version:1.42.0
Package:azureml-train-core, training version:1.43.0, current version:1.42.0
Package:azureml-train-restclients-hyperdrive, training v

Run(Experiment: automl_run,
Id: AutoML_459e811c-00f4-4099-a31f-a186a9f4de26_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('9', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.5, eta=0.5, gamma=0, max_depth=6, max_leaves=3, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'})

In [11]:
#TODO: Save the best model
joblib.dump(fitted_model,filename='automl_model.joblib')

['automl_model.joblib']

In [12]:
model_name = best_run.properties['model_name']
model_name

'AutoML459e811c038'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
env = best_run.get_environment().save_to_directory(path='environment')
script_file= 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

model = run.register_model(model_name = model_name)
print(model.name, model.id, model.version, sep='\t')

AutoML459e811c038	AutoML459e811c038:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
inference_config = InferenceConfig(entry_script=script_file, environment=env)

# create deployment config i.e. compute resources
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
)
service_name = "automlfire"
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-12 08:38:58+00:00 Creating Container Registry if not exists.
2022-08-12 08:38:59+00:00 Use the existing image.
2022-08-12 08:39:00+00:00 Submitting deployment to compute.
2022-08-12 08:39:04+00:00 Checking the status of deployment automlfire..
2022-08-12 08:41:42+00:00 Checking the status of inference endpoint automlfire.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [16]:
service.get_logs()

'2022-08-12T08:41:23,594911600+00:00 - rsyslog/run \n2022-08-12T08:41:23,611732400+00:00 - nginx/run \n2022-08-12T08:41:23,612742700+00:00 - iot-server/run \n2022-08-12T08:41:23,630934500+00:00 - gunicorn/run \n2022-08-12T08:41:23,640852300+00:00 | gunicorn/run | \n2022-08-12T08:41:23,657412600+00:00 | gunicorn/run | ###############################################\n2022-08-12T08:41:23,679728000+00:00 | gunicorn/run | AzureML Container Runtime Information\n2022-08-12T08:41:23,696098000+00:00 | gunicorn/run | ###############################################\n2022-08-12T08:41:23,700644300+00:00 | gunicorn/run | \n2022-08-12T08:41:23,707441200+00:00 | gunicorn/run | \n2022-08-12T08:41:23,714614500+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220708.v2\n2022-08-12T08:41:23,721708300+00:00 | gunicorn/run | \n2022-08-12T08:41:23,723674000+00:00 | gunicorn/run | \n2022-08-12T08:41:23,730508100+00:00 | gunicorn/run | PATH environment variable

In [17]:
service.update(enable_app_insights=True)

In [18]:
# send raw HTTP request to test the web service.

data = {"data": [{"day": 4, "month": 6, "year": 2012, "Temperature": 25, "RH": 89, "Ws": 13, "Rain": 2.5, "FFMC": 28.6, "DMC": 1.3, "DC": 6.9, "ISI": 0.0, "BUI": 1.7, "FWI": 0.0}]}
input_data = json.dumps(data)
                 
headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)
print(resp.json())

POST to url http://76fd5ea9-7b29-4e6c-a6e1-4c34cac6a03d.southcentralus.azurecontainer.io/score
prediction: "{\"result\": [0]}"
{"result": [0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
logs = service.get_logs()
print(logs)

2022-08-12T08:44:46,825972397+00:00 - rsyslog/run 
2022-08-12T08:44:46,826743388+00:00 - iot-server/run 
2022-08-12T08:44:46,844953674+00:00 - nginx/run 
2022-08-12T08:44:46,853084078+00:00 - gunicorn/run 
2022-08-12T08:44:46,859664900+00:00 | gunicorn/run | 
2022-08-12T08:44:46,878737976+00:00 | gunicorn/run | ###############################################
2022-08-12T08:44:46,884034113+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-08-12T08:44:46,908225828+00:00 | gunicorn/run | ###############################################
2022-08-12T08:44:46,934158023+00:00 | gunicorn/run | 
2022-08-12T08:44:46,959146929+00:00 | gunicorn/run | 
2022-08-12T08:44:46,973956354+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220708.v2
2022-08-12T08:44:46,981093170+00:00 | gunicorn/run | 
2022-08-12T08:44:46,989227275+00:00 | gunicorn/run | 
2022-08-12T08:44:46,991512048+00:00 | gunicorn/run | PATH environment variable: /azureml-env

In [21]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
